<a href="https://colab.research.google.com/github/AnnaZhuravleva/avtobreja/blob/master/why_so_bad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pickle

from collections import Counter

In [0]:
from scipy.stats import zscore

In [0]:
import pandas as pd

In [0]:
import numpy as np

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path = "/content/drive/My Drive/studies/HSE/prog/nlp/proj/sent_food.pickle"

In [0]:
with open(path, "rb") as f:
  c = pickle.load(f)

In [12]:
c[2]['пагубный_ADJ']

0.001698874868452549

In [8]:
!wget https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/word_lists/semantic_axis_method/service.csv

--2019-12-22 17:16:10--  https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/word_lists/semantic_axis_method/service.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1367659 (1.3M) [text/plain]
Saving to: ‘service.csv.1’

service.csv.1       100%[===================>]   1.30M  8.31MB/s    in 0.2s    

2019-12-22 17:16:10 (8.31 MB/s) - ‘service.csv.1’ saved [1367659/1367659]



In [0]:
with open ("service.csv", "r") as f:
  lines = f.readlines()

le = Counter()
for line in lines:
  w, count = line.strip().split()
  le[w] = float(count)

c.append(le)

In [14]:
for i in range(5):
  print(i)
  print(max(dict(c[i]).values()))
  print(min(dict(c[i]).values()))

0
132.83712725763326
-357.5816257435363
1
4686.967115458101
-947.8087743571377
2
1150.003318358924
-67.35316430628018
3
40272.254946324974
-451.7032948601991
4
1.2989414930343628
0.3453672528266907


In [15]:
word = 'вкусный_ADJ'
for i in range(5):
  print(i)
  print(c[i][word])

0
-20.492670276609715
1
3201.9742432385683
2
1010.6613312746795
3
32946.22784128785
4
0.5756369531154633


In [33]:
vocab = set(c[0])
print(len(vocab))
vocab.update(set(c[4]))
print(len(vocab))

101050
102766


In [98]:
counts = pd.DataFrame(columns=range(5), index=sorted(vocab))
counts.head()

,0,1,2,3,4
!!_PUNCT,NaN,NaN,NaN,NaN,NaN
!&quot_X,NaN,NaN,NaN,NaN,NaN
!(_PUNCT,NaN,NaN,NaN,NaN,NaN
!)_PUNCT,NaN,NaN,NaN,NaN,NaN
!........._PUNCT,NaN,NaN,NaN,NaN,NaN


In [0]:
for i in range(5):
  counts[i] = [el[1] for el in sorted(list(dict(c[i]).items()) + [(el, 0.0) for el in vocab.difference(set(c[i]))], key=lambda x: x[0])]

In [100]:
counts.tail()

,0,1,2,3,4
№№_SYM,0.0,0.0,0.0,0.0,0.0
●_SYM,0.0,0.0,0.0,0.0,0.0
☺_SYM,0.0,0.0,0.0,0.0,0.0
♡♡♡_SYM,0.0,0.0,0.0,0.0,0.0
_NOUN,0.0,0.0,0.0,0.0,0.0


In [0]:
counts['zero'] = np.sum([counts[0], counts[1], counts[2], counts[3], counts[4]], axis=0)

In [0]:
from scipy.stats import logistic

In [102]:
counts = counts[counts['zero'] > 0].drop(['zero'], axis=1)
counts.tail()

,0,1,2,3,4
“отжимают”_NOUN,-0.005490,0.0,0.0,2.472265,0.0
“приложиться_VERB,-0.036891,0.0,0.0,0.860026,0.0
“стейк_PROPN,-0.033231,0.0,0.0,0.978386,0.0
”прошелся”_NOUN,0.097024,0.0,0.0,4.334878,0.0
…_PUNCT,-3.165581,0.0,0.0,17.262491,0.0


In [0]:
words = ['понравиться_VERB', 'вкусный_ADJ', 'травмоопасный_ADJ', 'неухоженный_ADJ', 'сумасшедший_ADJ', 'пагубный_ADJ', 'издевательский_ADJ', 'противность_NOUN', 'давка_NOUN']

In [88]:
counts = counts.apply(lambda x: logistic.cdf(x))
counts.tail()

,0,1,2,3,4
ять_NOUN,0.622753,0.620154,0.617661,0.616980,0.658449
ять_VERB,0.622753,0.620154,0.617661,0.616980,0.661309
ячейка_NOUN,0.624680,0.621130,0.618472,0.618359,0.635659
ящерица_NOUN,0.622753,0.620154,0.617661,0.616980,0.646044
ящик_NOUN,0.630842,0.624902,0.625858,0.626750,0.651882


In [103]:
for word in words:
  if word in counts.index:
    print(counts.loc[word])
  else:
    print(word)
    print("not found")

0      -42.654162
1      884.800879
2      317.989657
3    12288.315600
4        0.621756
Name: понравиться_VERB, dtype: float64
0      -20.492670
1     3201.974243
2     1010.661331
3    32946.227841
4        0.575637
Name: вкусный_ADJ, dtype: float64
0   -0.005373
1   -0.001391
2   -0.000090
3   -0.500831
4    0.858421
Name: травмоопасный_ADJ, dtype: float64
неухоженный_ADJ
not found
0    -0.051934
1     0.308550
2     1.013306
3    54.791854
4     1.122154
Name: сумасшедший_ADJ, dtype: float64
0    0.259198
1   -0.427478
2    0.001699
3    7.408468
4    1.015671
Name: пагубный_ADJ, dtype: float64
0     0.567685
1     0.663180
2     0.143479
3    12.944970
4     1.029322
Name: издевательский_ADJ, dtype: float64
0    0.000000
1    0.000000
2    0.000000
3    0.000000
4    1.045236
Name: противность_NOUN, dtype: float64
давка_NOUN
not found
